In [ ]:
#!pip install -U ddgs
# %pip install -U langchain-google-vertexai
# %pip install --upgrade langchain-core


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
import os, getpass
from langchain_google_genai import ChatGoogleGenerativeAI #BaseChatModel
load_dotenv()  # Loads environment variables from .env file
# ignore all warnings in this notebook
import warnings
warnings.filterwarnings("ignore")

/workspaces/Langchain-Tutorial/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.agents import create_agent
from langchain.tools import tool

In [3]:
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("GOOGLE_API_KEY found in .env file.")
else:
    print("GOOGLE_API_KEY not found in .env file.")
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter GEMINI API key: ")

GOOGLE_API_KEY not found in .env file.


In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

E0000 00:00:1765363767.300886    1681 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [25]:
# Tools Creation
from langchain.tools import tool
from datetime import datetime
import random

@tool
def search_web(query: str) -> str:
    """Search the web for current information.
    
    Args:
        query: The search query string
        
    Returns:
        Search results as a string
    """
    # In production, you'd use a real search API
    # For demo purposes, we'll simulate results
    simulated_results = {
        "weather": "Current weather: Sunny, 72°F in San Francisco",
        "news": "Latest AI news: New GPT model released with improved reasoning",
        "stocks": "Tech stocks up 2.5% today, AI sector leading gains"
    }
    
    for key in simulated_results:
        if key in query.lower():
            return simulated_results[key]
    
    return f"Search results for '{query}': Found 10 relevant articles..."


@tool
def calculator(expression: str) -> str:
    """Perform mathematical calculations.
    
    Args:
        expression: A mathematical expression to evaluate
        
    Returns:
        The calculation result
    """
    try:
        # Safe evaluation of mathematical expressions
        result = eval(expression, {"__builtins__": {}}, {})
        return f"Result: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"


@tool
def get_current_time(timezone: str = "UTC") -> str:
    """Get the current time in a specific timezone.
    
    Args:
        timezone: The timezone (default: UTC)
        
    Returns:
        Current time as a string
    """
    current_time = datetime.now()
    return f"Current time in {timezone}: {current_time.strftime('%Y-%m-%d %H:%M:%S')}"

In [9]:
# Get all tool metadata
tools=[search_web, calculator, get_current_time]
for tool_obj in tools:
    print(f"Tool: {tool_obj.name}")
    print(f"  Description: {tool_obj.description}")
    print(f"  Input schema: {tool_obj.args_schema}")
    print()

Tool: search_web
  Description: Search the web for current information.

    Args:
        query: The search query string

    Returns:
        Search results as a string
  Input schema: <class 'langchain_core.utils.pydantic.search_web'>

Tool: calculator
  Description: Perform mathematical calculations.

    Args:
        expression: A mathematical expression to evaluate

    Returns:
        The calculation result
  Input schema: <class 'langchain_core.utils.pydantic.calculator'>

Tool: get_current_time
  Description: Get the current time in a specific timezone.

    Args:
        timezone: The timezone (default: UTC)

    Returns:
        Current time as a string
  Input schema: <class 'langchain_core.utils.pydantic.get_current_time'>



In [26]:
# Agent Creation
# Create the agent with tools
agent = create_agent(
    model=model,
    tools=[search_web, calculator, get_current_time],
    system_prompt="""You are a helpful AI assistant with access to tools.
    
    You can:
    - Search the web for current information
    - Perform calculations
    - Get the current time
    
    Think step-by-step and use tools when needed to provide accurate answers.
    Be concise but thorough."""
)

In [27]:
# Example 1: Simple tool use
def run_agent(user_message: str):
    """Run the agent with a user message."""
    print(f"\n{'='*60}")
    print(f"USER: {user_message}")
    print(f"{'='*60}\n")
    
    result = agent.invoke({
        "messages": [{"role": "user", "content": user_message}]
    })
    
    # Print the agent's response
    final_message = result["messages"][-1]
    print(f"AGENT: {final_message.content}\n")
    
    return result

In [29]:
# Test 1: Time query
run_agent("What time is it ?")


USER: What time is it ?

AGENT: The current time is 10:51:22 UTC on December 10, 2025.



{'messages': [HumanMessage(content='What time is it ?', additional_kwargs={}, response_metadata={}, id='4f9f4ea4-28b8-49c1-8d26-15deb01735bc'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_time', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b07e3-1538-7b42-96ab-547ab39a58bb-0', tool_calls=[{'name': 'get_current_time', 'args': {}, 'id': '252d2f28-39fa-4956-8723-63e314e9fadc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 60, 'total_tokens': 319, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 48}}),
  ToolMessage(content='Current time in UTC: 2025-12-10 10:51:22', name='get_current_time', id='1d631408-17fb-49e1-a1c8-71faa8b21e36', tool_call_id='252d2f28-39fa-4956-8723-63e314e9fa

In [ ]:
# Test 2: Calculation
run_agent("If I buy 3 items at $24.99 each and have a 15% discount, what's my total?")



USER: If I buy 3 items at $24.99 each and have a 15% discount, what's my total?

AGENT: Your total would be $63.72.



{'messages': [HumanMessage(content="If I buy 3 items at $24.99 each and have a 15% discount, what's my total?", additional_kwargs={}, response_metadata={}, id='7129eaf7-31c3-4130-af97-df44f68d8e94'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'calculator', 'arguments': '{"expression": "(3 * 24.99) * 0.85"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b07ec-c6a2-7380-95e3-edd2526b9165-0', tool_calls=[{'name': 'calculator', 'args': {'expression': '(3 * 24.99) * 0.85'}, 'id': '8b0186fb-66fb-452a-8489-f5092d447cf2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 283, 'output_tokens': 205, 'total_tokens': 488, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 178}}),
  ToolMessage(content='Result: 63.7245', name='calculator', id='67d7

In [ ]:
# Test 3: Web search
run_agent("What's the weather like today in San Fransico?")

In [20]:
# Test 3: Multi-step reasoning
run_agent("Search for AI news, then calculate how many days until the end of the year")


USER: Search for AI news, then calculate how many days until the end of the year

AGENT: AI news: A new GPT model has been released with improved reasoning capabilities. There are 21 days until the end of the year.



{'messages': [HumanMessage(content='Search for AI news, then calculate how many days until the end of the year', additional_kwargs={}, response_metadata={}, id='e729bbab-513a-4908-9750-fe11ae3ac75c'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_time', 'arguments': '{"timezone": "America/New_York"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b07db-d7ee-7631-9f0d-2afe31c090f2-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'AI news'}, 'id': '1c470773-3356-4ced-b7d5-0041249702c8', 'type': 'tool_call'}, {'name': 'get_current_time', 'args': {'timezone': 'America/New_York'}, 'id': 'de8935e3-d580-49d5-ba29-a283a4214991', 'type': 'tool_call'}], usage_metadata={'input_tokens': 270, 'output_tokens': 37, 'total_tokens': 307, 'input_token_details': {'c

In [ ]:
# Test 3: Multi-step reasoning
run_agent("Search for AI news, then calculate how many days until the end of the year")


USER: Search for AI news, then calculate how many days until the end of the year

AGENT: [{'type': 'text', 'text': 'Latest AI news: New GPT model released with improved reasoning.\n\nThere are 22 days until the end of the year.', 'extras': {'signature': 'CsUDAXLI2nzktJ0uTLOEXb2RqV1t97+cfHuWONA019uF6KO/Lkmw4Mvls97wKoyOg1iXGP9yx17Z7jM2EMaL0NvjhYzYtkovBc2sRhQ6f8dEc0LlJm/jdynnndjPU35DwNPlCqdPoH9NeAzVXYJwuKnxjoQ9vSlBtHKT/rT1HGXDmRW/HCFlCJKK8ygr/avn+AySGOx1tMAxVE6lY9FeA46hzptVZXNeHAMIBTXbXdZKzZMwM28zLGcAqBTu0JYqi8Ngl/eVzg6UMS39lhWunjv72REnraYlCT/HKd/kLXmKwC2gYXZ4JoGV4sjJOm5xb3FkoCcWgw/PMXx1GfMCH7ha+dsRvREpe4/wOaHR+tGloo5HcDSWnv179cvx9i6FNeMZ0djmTlDM6+P2dP2bRKwUlH3KJnHet/KVwyBYbd9hlC2NMiIuX9iC5mt7auriW9vx1vQuUif+bBAWCOzEnWU5qgWi22A/Lwm/fFLa6k9n6m3zv/0vZVpuQG6iLWbB4kQ856AjruLOMz15KnjhFbWKHGK9SxktMn4Q9u18mixw4YClye+gvu+O3TFqkk3sQtlbd6rqwSl63QKiuAQHJxY/cCcUmXLt'}}]



{'messages': [HumanMessage(content='Search for AI news, then calculate how many days until the end of the year', additional_kwargs={}, response_metadata={}, id='56439c0c-c874-405d-863b-bcae21e3988a'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_web', 'arguments': '{"query": "AI news"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--019b0395-74e6-7163-a952-0d5f89761282-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'AI news'}, 'id': '757dceff-8585-4a60-aa76-caeddf298540', 'type': 'tool_call'}], usage_metadata={'input_tokens': 270, 'output_tokens': 149, 'total_tokens': 419, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 133}}),
  ToolMessage(content='Latest AI news: New GPT model released with improved reasoning', name='searc

In [21]:
# Streaming

# Streaming example
print("\n" + "="*60)
print("STREAMING DEMO: Watch the agent think!")
print("="*60 + "\n")

for chunk in agent.stream({
    "messages": [{"role": "user", "content": "Calculate 15% of 299, then tell me the time"}]
}, stream_mode="values"):
    latest_message = chunk["messages"][-1]
    
    if hasattr(latest_message, 'content') and latest_message.content:
        print(f"💭 Agent: {latest_message.content}")
    elif hasattr(latest_message, 'tool_calls') and latest_message.tool_calls:
        tool_names = [tc['name'] for tc in latest_message.tool_calls]
        print(f"🔧 Using tools: {', '.join(tool_names)}")


STREAMING DEMO: Watch the agent think!

💭 Agent: Calculate 15% of 299, then tell me the time
🔧 Using tools: calculator, get_current_time
💭 Agent: Current time in UTC: 2025-12-10 10:45:42
💭 Agent: 15% of 299 is 44.85. The current time is 10:45:42 UTC.


In [ ]:
# Middleware

In [53]:
# Tool error handling
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage
@tool
def divide_numbers(a: float, b: float) -> str:
    """Divide two numbers. Will fail if dividing by zero.
    
    Args:
        a: The numerator
        b: The denominator
    """
    result = a / b  # This will raise ZeroDivisionError if b=0
    return f"Result: {result}"

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Gracefully handle tool failures."""
    try:
        return handler(request)
    except Exception as e:
        return ToolMessage(
            content=f"Tool failed: {str(e)}. Please try a different approach.",
            tool_call_id=request.tool_call["id"]
        )

# Add error handling to agent
agent_with_errors = create_agent(
    model=model,
    tools=[divide_numbers],
    middleware=[handle_tool_errors],
    system_prompt="You are a helpful assistant..."
)

In [ ]:
agent_with_errors.invoke({"messages": [{"role": "user", "content": "2/0"}]})


In [ ]:
# Dynamic Model Selection

In [ ]:
# Dynamic model
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse


basic_model = ChatOpenAI(model="gpt-4o-mini")
advanced_model = ChatOpenAI(model="gpt-4o")

@wrap_model_call
def smart_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Use basic model for simple queries, advanced for complex ones."""
    message_count = len(request.state["messages"])
    
    # Use advanced model for longer conversations
    if message_count > 5:
        request.model = advanced_model
    else:
        request.model = basic_model
    
    return handler(request)

agent_smart = create_agent(
    model=basic_model,
    tools=[search_web, calculator, get_current_time],
    middleware=[smart_model_selection]
)

In [ ]:
# Dyanmic System Prompt 

In [ ]:
# Dynamic system prompt
from typing import TypedDict

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_role = request.runtime.context.get("user_role", "user")
    base_prompt = "You are a helpful assistant."

    if user_role == "expert":
        return f"{base_prompt} Provide detailed technical responses."
    elif user_role == "beginner":
        return f"{base_prompt} Explain concepts simply and avoid jargon."

    return base_prompt


@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role and available tools."""
    user_role = request.runtime.context.get("user_role", "user")
    tools_list = ", ".join([tool.__name__ for tool in request.runtime.tools])
    
    base_prompt = f"You are a helpful assistant with access to tools: {tools_list}."

    if user_role == "expert":
        return f"{base_prompt} Provide detailed technical responses and leverage tools effectively."
    elif user_role == "beginner":
        return f"{base_prompt} Explain concepts simply, avoid jargon, and use tools only when necessary."

    return base_prompt

agent = create_agent(
    model=model,
    tools=[search_web, calculator, get_current_time],
    middleware=[user_role_prompt],
    context_schema=Context
)

# The system prompt will be set dynamically based on context
result = agent.invoke(
    {"messages": [{"role": "user", "content": "If I buy 3 items at $24.99 each and have a 15% discount, what's my total?"}]},
    context={"user_role": "expert"}
)